# exp064  
2.5D全臓器学習用ラベルの作成.  
extravasation: ian pan bboxラベルをラベルとする.  
bowel: 元データセットのラベルを使用.  
lsk: セグメンテーションの結果そのスライスにその臓器が存在するかつ、patient levelのlabelが1の場合1とする.  

In [ ]:
import os
import random
import sys
import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict
from typing import Tuple

import albumentations as A
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.cuda.amp as amp
from sklearn.model_selection import GroupKFold
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader
from tqdm import tqdm
import timm

# リポジトリtopに移動
while os.path.basename(os.getcwd()) != 'rsna-2023':
    os.chdir('../')
    if os.getcwd() == '/':
        raise Exception('Could not find project root directory.')

from src.classification.dataset import TrainDatasetBowelExtra as TrainDataset
from src.classification.dataset import save_df, load_df
from src.image_processing import windowing
from src.logger import get_logger
from src.classification.model import lock_model_encoder_weight
from src.classification.trainer import fit_model

In [ ]:
class CFG:
    exp_name = 'exp_064'
    # path
    image_dir = "data/dataset001"
    model_save_dir = "outputs"
    # other config
    seed = 42
    num_workers = 4
    num_gpus = 2
    progress_bar = True